In [ ]:
!pip install pypdf --upgrade

In [ ]:
!python -m spacy download es_core_news_sm

In [161]:
with open('Capítulo Uno.txt', 'r', encoding='utf-8-sig') as archivo:
      texto = archivo.read()

In [162]:
texto_separado = re.split(r'\n',texto)

anotaciones = []

for text in texto_separado:
  dato = re.findall('\".+\"', text)
  if dato:
    etiqueta = "conversacion"
  else:
    etiqueta = "contexto"

  anotaciones.append({"texto": text, "etiqueta": etiqueta})

In [ ]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [220]:
from transformers import pipeline


classifier = pipeline("text-classification", model="nmarinnn/bert-bregman")

config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [221]:
classifier("Odio amar a mi mamá")

[{'label': 'negative', 'score': 0.9481379389762878}]

In [222]:
conversacion = []

for text in anotaciones:
  contexto_texto = text['texto'].split('-')
  quien = text['etiqueta']
  if len(contexto_texto) > 1:
    for i in [num for num in range(len(contexto_texto)) if num % 2 != 0]:
      sentencia_para_verbo = nlp(contexto_texto[i])
      for token in sentencia_para_verbo:
        if token.pos_ == "VERB" and token.dep_ == "ROOT" and 'ó' in token.text:
          quien = "otro"
        elif token.dep_ == "ROOT":
          quien = "yo"
  if quien == "conversacion":
    size_backwards = len(conversacion)
    while size_backwards > 0:
      if conversacion[size_backwards - 1]["tipo"] == "otro":
        quien = "yo"

        break
      elif conversacion[size_backwards - 1]["tipo"] == "yo":
        quien = "otro"
        break
      size_backwards -= 1
  emotion = classifier(text['texto'])
  conversacion.append({"texto": text['texto'], "tipo": quien, "emocion":emotion})

In [223]:
conversacion

[{'texto': 'Capítulo Uno: Con el pie izquierdo',
  'tipo': 'contexto',
  'emocion': [{'label': 'neutral', 'score': 0.491995632648468}]},
 {'texto': 'Enero de 1909, Ciudad de México, México.',
  'tipo': 'contexto',
  'emocion': [{'label': 'neutral', 'score': 0.6452159881591797}]},
 {'texto': 'Aquel día, a inicios de año, supe que había que enterrar a mis padres. No pude enterarme a través de las constantes cartas que nos enviábamos durante su estancia en los Estados Unidos. Sino al abogado de la familia que me citó en la capital para tratar asuntos importantes.',
  'tipo': 'contexto',
  'emocion': [{'label': 'positive', 'score': 0.4707798659801483}]},
 {'texto': 'Los viajes de negocios eran muy habituales en ellos, siempre iban juntos y siempre volvían juntos para irse de nuevo por una temporada más y así. Se suponía que volverían de nuevo.',
  'tipo': 'contexto',
  'emocion': [{'label': 'positive', 'score': 0.7929803729057312}]},
 {'texto': '¿Cuándo fue la última vez que los vi?',
  't

In [224]:
# prompt: export dictionary as json

import json

# Assuming 'conversacion' is the dictionary you want to export
with open('conversacion.json', 'w', encoding='utf-8') as f:
  json.dump(conversacion, f, ensure_ascii=False, indent=4)